In [ ]:
%matplotlib inline

import ast
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator
from matplotlib.ticker import ScalarFormatter
from matplotlib.patches import Patch
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Arial'
import matplotlib.gridspec as gridspec

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as ticker
import matplotlib
import seaborn as sns
from itertools import groupby
from scipy import integrate
from tqdm import tqdm
import cartopy.crs as ccrs
import cartopy


plt.style.use('ggplot')
pd.set_option('chained_assignment',None)

sys.path.append(os.path.join( '..'))
from gmtra.utils import load_config,sum_tuples,monetary_risk,set_prot_standard,pluvial_design,pluvial_design_1up,gdp_lookup,get_value,get_mean,wbregion

data_path = load_config()['paths']['data']

In [ ]:
%%time
global_countries = gpd.read_file(os.path.join(data_path,'input_data','global_countries.shp'))
wbccodes = pd.read_csv(os.path.join(data_path,'input_data','wbccodes2014.csv'))
global_regions = gpd.read_file(os.path.join(data_path,'input_data','global_regions_v2.shp'))
prot_lookup = dict(zip(global_regions['GID_2'],global_regions['prot_stand']))
ne_countries = gpd.read_file(os.path.join(data_path,'input_data','ne_50m_admin_0_countries.shp'))
incomegroups = pd.read_csv(os.path.join(data_path,'input_data','incomegroups_2018.csv'))
incomegroups.GroupCode = incomegroups.GroupCode.apply(lambda x: x.upper())

In [ ]:
wbc_lookup = dict(zip(incomegroups.CountryCode,incomegroups.GroupCode))

In [ ]:
%%time
events_FU = ['FU-5', 'FU-10', 'FU-20', 'FU-50', 'FU-75', 'FU-100', 'FU-200', 'FU-250','FU-500', 'FU-1000']
tot_road_FU = pd.read_csv(os.path.join(data_path,'summarized','FU_road_losses.csv'),
                          converters = dict(zip(events_FU,[ast.literal_eval]*len(events_FU))),index_col=[0]) 
print('FU loaded')

events_PU = ['PU-5', 'PU-10', 'PU-20', 'PU-50', 'PU-75', 'PU-100', 'PU-200', 'PU-250', 'PU-500', 'PU-1000']
tot_road_PU = pd.read_csv(os.path.join(data_path,'summarized','PU_road_losses.csv'),
                          converters = dict(zip(events_PU,[ast.literal_eval]*len(events_PU))),index_col=[0]) 
print('PU loaded')

events_CF = ['CF-10', 'CF-20', 'CF-50', 'CF-100', 'CF-200', 'CF-500', 'CF-1000']
tot_road_CF = pd.read_csv(os.path.join(data_path,'summarized','CF_road_losses.csv'),
                          converters = dict(zip(events_CF,[ast.literal_eval]*len(events_CF))),index_col=[0])

In [ ]:
tot_road_FU = tot_road_FU.merge(incomegroups,left_on='country',right_on='CountryCode').merge(global_countries[['ISO_3digit','wbregion']],left_on='country',right_on='ISO_3digit')
tot_road_PU = tot_road_PU.merge(incomegroups,left_on='country',right_on='CountryCode').merge(global_countries[['ISO_3digit','wbregion']],left_on='country',right_on='ISO_3digit')
tot_road_CF = tot_road_CF.merge(incomegroups,left_on='country',right_on='CountryCode').merge(global_countries[['ISO_3digit','wbregion']],left_on='country',right_on='ISO_3digit')

In [ ]:
%%time
tqdm.pandas()
tot_road_FU_base = tot_road_FU.progress_apply(lambda x : set_prot_standard(x,prot_lookup,events_FU),axis=1)
tot_road_CF_base = tot_road_CF.progress_apply(lambda x : set_prot_standard(x,prot_lookup,events_CF),axis=1)
tot_road_FU_base = tot_road_FU_base.progress_apply(lambda x : pluvial_design(x,'FU'),axis=1)
tot_road_CF_base = tot_road_CF_base.progress_apply(lambda x : pluvial_design(x,'CF'),axis=1)
tot_road_PU_base = tot_road_PU.progress_apply(lambda x : pluvial_design(x,'PU'),axis=1)

In [ ]:
%%time
tqdm.pandas()
tot_road_FU_1up = tot_road_FU.progress_apply(lambda x : set_prot_standard(x,prot_lookup,events_FU),axis=1)
tot_road_CF_1up = tot_road_CF.progress_apply(lambda x : set_prot_standard(x,prot_lookup,events_CF),axis=1)
tot_road_FU_1up = tot_road_FU_1up.progress_apply(lambda x : pluvial_design_1up(x,'FU'),axis=1)
tot_road_CF_1up = tot_road_CF_1up.progress_apply(lambda x : pluvial_design_1up(x,'CF'),axis=1)
tot_road_PU_1up = tot_road_PU.progress_apply(lambda x : pluvial_design_1up(x,'PU'),axis=1)

In [ ]:
%%time
FU_reg_stats = tot_road_FU_base.loc[tot_road_FU.road_type.isin(['primary','secondary','tertiary'])]
FU_reg_stats = FU_reg_stats.groupby(['region','road_type'])[events_FU].agg(sum_tuples)

PU_reg_stats = tot_road_PU_base.loc[tot_road_PU.road_type.isin(['primary','secondary','tertiary'])]
PU_reg_stats = PU_reg_stats.groupby(['region','road_type'])[events_PU].agg(sum_tuples)

CF_reg_stats = tot_road_CF_base.loc[tot_road_CF.road_type.isin(['primary','secondary','tertiary'])]
CF_reg_stats = CF_reg_stats.groupby(['region','road_type'])[events_CF].agg(sum_tuples)

FU_reg_stats_1up = tot_road_FU_1up.loc[tot_road_FU.road_type.isin(['primary','secondary','tertiary'])]
FU_reg_stats_1up= FU_reg_stats_1up.groupby(['region','road_type'])[events_FU].agg(sum_tuples)

PU_reg_stats_1up = tot_road_PU_1up.loc[tot_road_PU.road_type.isin(['primary','secondary','tertiary'])]
PU_reg_stats_1up = PU_reg_stats_1up.groupby(['region','road_type'])[events_PU].agg(sum_tuples)

CF_reg_stats_1up = tot_road_CF_1up.loc[tot_road_CF.road_type.isin(['primary','secondary','tertiary'])]
CF_reg_stats_1up = CF_reg_stats_1up.groupby(['region','road_type'])[events_CF].agg(sum_tuples)

In [ ]:
%%time
tqdm.pandas()
RPS = [1/5,1/10,1/20,1/50,1/75,1/100,1/200,1/250,1/500,1/1000]
reg_risk_PU = pd.DataFrame(PU_reg_stats.progress_apply(lambda x: monetary_risk(x,RPS,events_PU),axis=1).tolist(),index=PU_reg_stats.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])
reg_risk_PU_1up = pd.DataFrame(PU_reg_stats_1up.progress_apply(lambda x: monetary_risk(x,RPS,events_PU),axis=1).tolist(),index=PU_reg_stats_1up.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])

reg_risk_FU = pd.DataFrame(FU_reg_stats.progress_apply(lambda x: monetary_risk(x,RPS,events_FU),axis=1).tolist(),index=FU_reg_stats.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])
reg_risk_FU_1up = pd.DataFrame(FU_reg_stats_1up.progress_apply(lambda x: monetary_risk(x,RPS,events_FU),axis=1).tolist(),index=FU_reg_stats_1up.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])


RPS = [1/10,1/20,1/50,1/100,1/200,1/500,1/1000]
reg_risk_CF = pd.DataFrame(CF_reg_stats.progress_apply(lambda x: monetary_risk(x,RPS,events_CF),axis=1).tolist(),
                       index=CF_reg_stats.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])
reg_risk_CF_1up = pd.DataFrame(CF_reg_stats_1up.progress_apply(lambda x: monetary_risk(x,RPS,events_CF),axis=1).tolist(),
                       index=CF_reg_stats.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])

In [ ]:
incomegroups = pd.read_csv(os.path.join(data_path,'input_data','incomegroups_2018.csv'),index_col=[0])
tot_road = pd.read_csv(os.path.join(data_path,'summarized','total_exposure_100prot.csv'))
tot_road = tot_road.merge(incomegroups,left_on='country',right_on='CountryCode')
tot_road = tot_road.loc[tot_road.GroupCode.isin(['LIC','HIC','UMC','LMC'])]
#tot_road = tot_road.loc[tot_road.road_type.isin(['primary','secondary','tertiary'])]

all_road_stats = pd.read_feather(os.path.join(data_path,'summarized','all_road_stats.ft'))
all_road_stats = pd.DataFrame(all_road_stats.groupby(['region','road_type']).sum().max(axis=1),columns=['all_risk_road'])
tot_road_inb = tot_road.drop([x for x in tot_road.columns if ('EQ' in x) | ('Cyc' in x)],axis='columns')
region_risk_road = pd.DataFrame(tot_road_inb.groupby(['region','road_type']).sum().sum(axis=1),columns=['tot_risk_road'])
region_risk_road = region_risk_road.merge(all_road_stats,left_index=True,right_index=True)
region_risk_road['rel_risk'] = (region_risk_road.tot_risk_road/region_risk_road.all_risk_road)

In [ ]:
total_infra_value = pd.read_csv(os.path.join(data_path,'summarized','total_infrastructure_values.csv'))

In [ ]:
total_infra_value = total_infra_value.groupby(['region','road_type']).sum()

In [ ]:
discount_rates = {'LIC': 0.12,'LMC' : 0.08,'UMC' : 0.05,'HIC' : 0.03}
rout_main = 0.0075
per_main = 0.05
life_span_road = {'primary': 20,'secondary' : 15,'tertiary' : 6}
upgrade_cost = 0.02

In [ ]:
def get_npv_cost(x):
    
    discount_rate = discount_rates[x.wbincome]
    y = x.infra_value
    
    return np.npv(discount_rate,[y*rout_main if (x%6) != 0 else y*(rout_main+per_main) for x in range(1,life_span_road[x.road_type])])

def get_npv_loss(x):
    
    discount_rate = discount_rates[x.wbincome]
    y = x.perc_50
    
    return np.npv(discount_rate,[y]*life_span_road[x.road_type])

In [ ]:
avoided_FU = pd.DataFrame((reg_risk_FU-reg_risk_FU_1up)['perc_50'])
avoided_FU = avoided_FU.merge(pd.DataFrame(region_risk_road['rel_risk']),left_index=True,right_index=True)
avoided_FU = avoided_FU.merge(total_infra_value,left_index=True,right_index=True)
avoided_FU = avoided_FU.reset_index(level=1).merge(global_regions[['GID_2','ISO_3digit','wbincome']].set_index('GID_2'),
                                      left_index=True,right_index=True)
avoided_FU.infra_value = avoided_FU.infra_value*avoided_FU.rel_risk
avoided_FU['wbincome'] = avoided_FU.ISO_3digit.apply(lambda x : wbc_lookup[x])
avoided_FU['main'] = avoided_FU.apply(lambda x: get_npv_cost(x),axis=1)
avoided_FU['upgrade'] = avoided_FU.infra_value*upgrade_cost
avoided_FU['npv_loss'] = avoided_FU.apply(lambda y: get_npv_loss(y),axis=1)
avoided_FU['bca'] = ((avoided_FU['npv_loss']/(avoided_FU['main']+avoided_FU['upgrade'])))
avoided_FU['nb'] = ((avoided_FU['npv_loss']-(avoided_FU['main']+avoided_FU['upgrade'])))
avoided_FU['bca'][avoided_FU['bca'] > 10] = 10

In [ ]:
avoided_PU = pd.DataFrame((reg_risk_PU-reg_risk_PU_1up)['perc_50'])
avoided_PU = avoided_PU.merge(pd.DataFrame(region_risk_road['rel_risk']),left_index=True,right_index=True)
avoided_PU = avoided_PU.merge(total_infra_value,left_index=True,right_index=True)
avoided_PU = avoided_PU.reset_index(level=1).merge(global_regions[['GID_2','ISO_3digit','wbincome']].set_index('GID_2'),
                                      left_index=True,right_index=True)
avoided_PU.infra_value = avoided_PU.infra_value*avoided_PU.rel_risk
avoided_PU['wbincome'] = avoided_PU.ISO_3digit.apply(lambda x : wbc_lookup[x])
avoided_PU['main'] = avoided_PU.apply(lambda x: get_npv_cost(x),axis=1)
avoided_PU['upgrade'] = avoided_PU.infra_value*upgrade_cost
avoided_PU['npv_loss'] = avoided_PU.apply(lambda y: get_npv_loss(y),axis=1)
avoided_PU['bca'] = ((avoided_PU['npv_loss']/(avoided_PU['main']+avoided_PU['upgrade'])))
avoided_PU['nb'] = ((avoided_PU['npv_loss']-(avoided_PU['main']+avoided_PU['upgrade'])))
avoided_PU['bca'][avoided_PU['bca'] > 10] = 10

In [ ]:
avoided_CF = pd.DataFrame((reg_risk_CF-reg_risk_CF_1up)['perc_50'])
avoided_CF = avoided_CF.merge(pd.DataFrame(region_risk_road['rel_risk']),left_index=True,right_index=True)
avoided_CF = avoided_CF.merge(total_infra_value,left_index=True,right_index=True)
avoided_CF = avoided_CF.reset_index(level=1).merge(global_regions[['GID_2','ISO_3digit','wbincome']].set_index('GID_2'),
                                      left_index=True,right_index=True)
avoided_CF.infra_value = avoided_CF.infra_value*avoided_CF.rel_risk
avoided_CF['wbincome'] = avoided_CF.ISO_3digit.apply(lambda x : wbc_lookup[x])
avoided_CF['main'] = avoided_CF.apply(lambda x: get_npv_cost(x),axis=1)
avoided_CF['upgrade'] = avoided_CF.infra_value*upgrade_cost
avoided_CF['npv_loss'] = avoided_CF.apply(lambda y: get_npv_loss(y),axis=1)
avoided_CF['bca'] = ((avoided_CF['npv_loss']/(avoided_CF['main']+avoided_CF['upgrade'])))
avoided_CF['nb'] = ((avoided_CF['npv_loss']-(avoided_CF['main']+avoided_CF['upgrade'])))
avoided_CF['bca'][avoided_CF['bca'] > 10] = 10

In [ ]:
avoided_all = pd.DataFrame((reg_risk_PU-reg_risk_PU_1up).add(reg_risk_FU-reg_risk_FU_1up,fill_value=0).add(reg_risk_CF-reg_risk_CF_1up,fill_value=0)['perc_50'])
avoided_all = avoided_all.merge(pd.DataFrame(region_risk_road[['rel_risk','all_risk_road']]),left_index=True,right_index=True)
avoided_all = avoided_all.merge(total_infra_value,left_index=True,right_index=True)
avoided_all = avoided_all.reset_index(level=1).merge(global_regions[['GID_2','ISO_3digit','wbincome']].set_index('GID_2'),
                                      left_index=True,right_index=True)
avoided_all.infra_value = avoided_all.infra_value# *avoided_all.rel_risk
avoided_all['wbincome'] = avoided_all.ISO_3digit.apply(lambda x : wbc_lookup[x])
avoided_all['main'] = avoided_all.apply(lambda x: get_npv_cost(x),axis=1)
avoided_all['upgrade'] = avoided_all.infra_value*upgrade_cost
avoided_all['npv_loss'] = avoided_all.apply(lambda y: get_npv_loss(y),axis=1)
avoided_all['bca'] = ((avoided_all['npv_loss']/(avoided_all['main']+avoided_all['upgrade'])))
avoided_all['nb'] = ((avoided_all['npv_loss']-(avoided_all['main']+avoided_all['upgrade'])))
avoided_all['bca'][avoided_all['bca'] > 10] = 10

In [ ]:
full_name = {'LIC':'Lower Income Countries','LMC':'Lower Middle Income Countries',
             'UMC':'Upper Middle Income Countries','HIC':'High Income Countries'}

save_results = []
for iter_,flood_type in enumerate(['Surface Flooding','River Flooding','Coastal Flooding','All Flooding']):
    for group in ['LIC','LMC','UMC','HIC']:
        for road_type in ['primary','secondary','tertiary']:
            if iter_ == 0:
                perc_bcr = len(avoided_PU['bca'].loc[(avoided_PU['bca'] > 1)  & (avoided_PU['road_type'] == road_type) & (
                    avoided_PU['wbincome'] == group)])/len(avoided_PU.loc[(avoided_PU['wbincome'] == group) & (avoided_PU['road_type'] == road_type)])*100
                net_benefit = avoided_PU['nb'].loc[(avoided_PU['bca'] > 1) & (avoided_PU['road_type'] == road_type) & (
                    avoided_PU['wbincome'] == group)].sum()/1e6
                avg_bca_pos = avoided_PU['bca'].loc[(avoided_PU['bca'] > 1)  & (avoided_PU['road_type'] == road_type) & (
                    avoided_PU['wbincome'] == group)].mean()
                avg_bca = avoided_PU['bca'].loc[(avoided_PU['road_type'] == road_type) & (avoided_PU['wbincome'] == group)].mean()                
            elif iter_ == 1:
                perc_bcr = len(avoided_FU['bca'].loc[(avoided_FU['bca'] > 1) &  (avoided_FU['road_type'] == road_type) & (
                    avoided_FU['wbincome'] == group)])/len(avoided_FU.loc[(avoided_FU['wbincome'] == group) & (avoided_FU['road_type'] == road_type)])*100            
                net_benefit = avoided_FU['nb'].loc[(avoided_FU['bca'] > 1)  &  (avoided_FU['road_type'] == road_type) & (
                    avoided_FU['wbincome'] == group)].sum()/1e6
                avg_bca_pos = avoided_FU['bca'].loc[(avoided_FU['bca'] > 1)  & (avoided_FU['road_type'] == road_type) & (
                    avoided_FU['wbincome'] == group)].mean()
                avg_bca = avoided_FU['bca'].loc[(avoided_FU['road_type'] == road_type) & (avoided_FU['wbincome'] == group)].mean()                

            elif iter_ == 2:
                perc_bcr = len(avoided_CF['bca'].loc[(avoided_CF['bca'] > 1) & (avoided_CF['road_type'] == road_type) & (
                    avoided_CF['wbincome'] == group)])/len(avoided_CF.loc[(avoided_CF['wbincome'] == group) & (avoided_CF['road_type'] == road_type)])*100            
                net_benefit = avoided_CF['nb'].loc[(avoided_CF['bca'] > 1) & (avoided_CF['road_type'] == road_type) & (
                    avoided_CF['wbincome'] == group)].sum()/1e6
                avg_bca_pos = avoided_CF['bca'].loc[(avoided_CF['bca'] > 1)  & (avoided_CF['road_type'] == road_type) & (
                    avoided_CF['wbincome'] == group)].mean()
                avg_bca = avoided_FU['bca'].loc[(avoided_FU['road_type'] == road_type) & (avoided_FU['wbincome'] == group)].mean()                
            else:
                perc_bcr = len(avoided_all['bca'].loc[(avoided_all['bca'] > 1) & (avoided_all['road_type'] == road_type) & (
                    avoided_all['wbincome'] == group)])/len(avoided_all.loc[(avoided_all['wbincome'] == group) & (avoided_all['road_type'] == road_type)])*100            
                net_benefit = avoided_all['nb'].loc[(avoided_all['bca'] > 1) & (avoided_all['road_type'] == road_type) & (
                    avoided_all['wbincome'] == group)].sum()/1e6
                avg_bca_pos = avoided_all['bca'].loc[(avoided_all['bca'] > 1)  & (avoided_all['road_type'] == road_type) & (
                    avoided_all['wbincome'] == group)].mean()
                avg_bca = avoided_FU['bca'].loc[(avoided_FU['road_type'] == road_type) & (avoided_FU['wbincome'] == group)].mean()                
            
            save_results.append([flood_type,round(perc_bcr,8),int(net_benefit),road_type,full_name[group],avg_bca_pos,avg_bca])

In [ ]:
bca_exposed = pd.DataFrame(save_results,columns=['flood','share_pos_BCR','Net_Benefit','road_type','wbincome','avg_pos_BCR','avg_BCR']).groupby(
    ['flood','road_type','wbincome']).sum().to_csv('first_results_bca_exposed.csv')

In [ ]:
avoided_all['all_risk_road'].loc[avoided_all.bca > 1].sum()/avoided_all['all_risk_road'].sum()*100

In [ ]:
bca_exposed = pd.read_csv('first_results_bca_exposed.csv',index_col=[0,1,2])
bca_exposed = bca_exposed.reindex(['Lower Income Countries', 'Lower Middle Income Countries', 
                           'Upper Middle Income Countries', 'High Income Countries'], level=2)
bca_all = pd.read_csv('first_results_bca.csv',index_col=[0,1,2])
bca_all = bca_all.reindex(['Lower Income Countries', 'Lower Middle Income Countries', 
                           'Upper Middle Income Countries', 'High Income Countries'], level=2)

In [ ]:
fig,axes = plt.subplots(2,3,figsize=(15,10),sharex=True)
flatui = ['#F3FFBD','#B2DBBF','#70C1B3','#247BA0']
floodtypes = ['Surface Flooding','River Flooding','Coastal Flooding']
for iter_,ax in enumerate(axes.flat):

    if iter_ < 3:
        bca_all_one = bca_all.loc[floodtypes[iter_]]
        sns.barplot(y = bca_all_one.share_pos_BCR,
                    x = bca_all_one.index.get_level_values(0),
                    hue= bca_all_one.index.get_level_values(1),
                    ax=ax, palette = sns.color_palette(flatui))
        
        ax.set_ylim(0,10)
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_title(floodtypes[iter_],fontweight='bold',fontsize=18,color='black')
    
    
    if iter_ > 2:
        bca_exposed_one = bca_exposed.loc[floodtypes[iter_-3]]
        sns.barplot(y = bca_exposed_one.share_pos_BCR,
                    x = bca_exposed_one.index.get_level_values(0),
                    hue= bca_exposed_one.index.get_level_values(1),
                    ax=ax, palette = sns.color_palette(flatui))
        
        ax.set_xlabel('Road Type',fontweight='bold',fontsize=16,color='black')
        ax.set_ylabel('')        
        ax.set_ylim(0,100)


    ax.tick_params(axis = 'both',labelcolor='black',color='black',labelsize=14) #
    ax.set_facecolor('#FAF9F9')
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')

    for y in range(12):
        ax.findobj(matplotlib.patches.Patch)[y].set_edgecolor('black')
    
    if iter_ > 0:
        ax.get_legend().remove()
    else:
        ax.set_ylabel('BCR > 1\nAll Roads (in %)',fontweight='bold',fontsize=16,color='black')
        legend_elements = [Patch(facecolor=flatui[0],edgecolor='black',linewidth=0.3,label='Low income'),
                          Patch(facecolor=flatui[1],edgecolor='black',linewidth=0.3,label='Lower middle income'),
                          Patch(facecolor=flatui[2],edgecolor='black',linewidth=0.3,label='Upper middle income'),
                          Patch(facecolor=flatui[3],edgecolor='black',linewidth=0.3,label='High income')]        #

        legend = ax.legend(handles=legend_elements,loc='upper left', shadow=True, 
                           fancybox=True,facecolor='#fefdfd',prop={'size':14})
        
    if iter_ == 3:
        ax.set_ylabel('BCR > 1\nExposed Roads (in %)',fontweight='bold',fontsize=16,color='black')
        

fig.tight_layout()
fig.savefig(os.path.join(data_path,'Figures','FigS7_CBA.png'),dpi=450)